In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Flatten, Dropout
from tensorflow.keras.layers import Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
import os


In [41]:
base_dir='data/cat_dog_fulls'
train_dir='data/cat_dog_fulls/train'
test_dir='data/cat_dog_fulls/test'
validation_dir='data/cat_dog_fulls/validation'

In [65]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.data import AUTOTUNE

# 데이터셋 경로 설정
base_dir = 'data/cat_dog_fulls'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# VGG16 모델 로드 (include_top=False로 설정하여 특성 맵만 사용)
model_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# 이미지를 로드하고 전처리하는 함수
def process_image(image_path):
    image = tf.io.read_file(image_path)  # 이미지 파일 읽기
    image = tf.image.decode_jpeg(image, channels=3)  # 이미지 디코딩
    image = tf.image.resize(image, (150, 150))  # 크기 변경
    image = tf.cast(image, tf.float32) / 255.0  # 정규화
    return image

# 레이블을 0 또는 1로 변환하는 함수
def get_label(file_path):
    parts = tf.strings.split(file_path, os.sep)
    label = parts[-2] == 'dogs'  # 'cats' -> 0, 'dogs' -> 1
    return label

# 이미지 파일 경로를 Tensor로 가져오는 함수
def get_image_paths_and_labels(directory):
    image_paths = []
    labels = []
    for class_name in ['cats', 'dogs']:
        class_dir = os.path.join(directory, class_name)
        for file_name in os.listdir(class_dir):
            image_paths.append(os.path.join(class_dir, file_name))
            labels.append(0 if class_name == 'cats' else 1)
    return image_paths, labels

# tf.data.Dataset을 사용하여 훈련 데이터셋을 생성
def create_dataset(image_paths, labels, batch_size):
    # tf.data.Dataset으로 이미지 경로와 레이블을 묶어서 반환
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

    # 이미지 전처리 및 배치 처리
    dataset = dataset.map(lambda x, y: (process_image(x), y), num_parallel_calls=AUTOTUNE)
    dataset = dataset.batch(batch_size).prefetch(AUTOTUNE)  # 성능 최적화
    return dataset

# 훈련, 검증, 테스트 데이터셋 생성
train_paths, train_labels = get_image_paths_and_labels(train_dir)
validation_paths, validation_labels = get_image_paths_and_labels(validation_dir)
test_paths, test_labels = get_image_paths_and_labels(test_dir)

batch_size = 32  # 배치 크기 설정
train_dataset = create_dataset(train_paths, train_labels, batch_size)
validation_dataset = create_dataset(validation_paths, validation_labels, batch_size)
test_dataset = create_dataset(test_paths, test_labels, batch_size)

# VGG16 모델을 통해 특징 추출하기
def extract_features_from_vgg(dataset, model_base):
    features = []
    labels = []
    
    for images, labels_batch in dataset:
        feature_batch = model_base.predict(images)  # VGG16을 이용한 특징 추출
        features.append(feature_batch)
        labels.append(labels_batch)
    
    return np.vstack(features), np.hstack(labels)

# 특징 추출
train_features, train_labels = extract_features_from_vgg(train_dataset, model_base)
validation_features, validation_labels = extract_features_from_vgg(validation_dataset, model_base)
test_features, test_labels = extract_features_from_vgg(test_dataset, model_base)

# 특징을 2D 배열로 변환 (4D -> 2D)
train_features = train_features.reshape(-1, 4 * 4 * 512)
validation_features = validation_features.reshape(-1, 4 * 4 * 512)
test_features = test_features.reshape(-1, 4 * 4 * 512)



1/1 [==============================] - 0s 212ms/step


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB4

input_shape=(360,360,3)
inputs=Input(shape=input_shape)

base_model=EfficientNetB4(include_top=False,
                         weights='imagenet',
                         input_tensor=inputs)
x=base_model.output
x=GlobalAveragePooling2D(x)
x=Dropuout(rate=0.3)(x)
x=Dense(units=128)(x)
x=Dropout(rate=0.3)(x)
x=Activation('relu')(x)
outputs=Dense(units=1,
              acativation='sigmoid')(x)
model=Model(optimizer=Adam(learning_rate=1e-3),
            loss='binary_crossentropy',
            metrics=['accuracy'])
model.fit(train_features, 
          train_labels, 
          epochs=10, 
          validation_data=(validation_features, validation_labels), 
          verbose=1)


